In [4]:
# Imports and Setup
from mido import MidiFile, MidiTrack, MetaMessage

In [5]:
# Define the input and output directories
input_file = '../songs/midi_songs/for those about to rock.mid'
output_file = '../songs/midi_songs/test-split.mid'

# Initialize MIDI files
in_mid = MidiFile(input_file)
out_mid = MidiFile(ticks_per_beat=in_mid.ticks_per_beat)

In [6]:
# Create drum track
drum_track = MidiTrack()
out_mid.tracks.append(drum_track)

# Collect messages with absolute times
all_messages = []
current_time = 0

# Process each track
for track in in_mid.tracks:
    current_time = 0
    for msg in track:
        current_time += msg.time
        # Check if the note is in the drum channel and append it to the new midi if it is
        if (isinstance(msg, MetaMessage) and msg.type in ('set_tempo', 'time_signature')) or \
            (msg.type in ('note_on', 'note_off') and msg.channel == 9) or \
            (msg.type not in ('note_on', 'note_off')):
            all_messages.append((current_time, msg))

# Sort and convert to delta times
all_messages.sort(key=lambda x: x[0])
last_time = 0

for abs_time, msg in all_messages:
    delta = abs_time - last_time
    new_msg = msg.copy(time=delta)
    drum_track.append(new_msg)
    last_time = abs_time

# Save and return
out_mid.save(output_file)